
CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [11]:
# load data from csv
# path for input and target data tables

diagnosis_path = '../data/test_phase_2/diagnosis_hadm.csv'
discharge_path ='../data/test_phase_2/discharge.csv'
edstays_path = '../data/test_phase_2/edstays.csv'
radiology_path = '../data/test_phase_2/radiology.csv'
triage_path = '../data/test_phase_2/triage.csv'
target_path = '../data/test_phase_2/discharge_target.csv'
discharge_sections_path = '../data/test_phase_2/discharge_sections.csv'
radiology_sections_path = '../data/test_phase_2/radiology_sections.csv'

import pandas as pd
import re
import os

# read data
diagnosis_df = pd.read_csv(diagnosis_path, keep_default_na=False)
discharge_df = pd.read_csv(discharge_path, keep_default_na=False)
edstays_df = pd.read_csv(edstays_path, keep_default_na=False)
radiology_df = pd.read_csv(radiology_path, keep_default_na=False)
triage_df = pd.read_csv(triage_path, keep_default_na=False)
target_df = pd.read_csv(target_path, keep_default_na=False)

discharge_sections_df = pd.read_csv(discharge_sections_path, keep_default_na=False)
radiology_sections_df = pd.read_csv(radiology_sections_path, keep_default_na=False)

In [21]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

# Initialize the LLM object with the specified model
model = LLM(model="meta-llama/Meta-Llama-3-8B-Instruct")
# model = LLM(model="meta-llama/Meta-Llama-3-8B")
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B')
# Imports
import torch
# generate_prompt function
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. 

### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
{instruction}

### Response:
"""

INFO 06-26 16:36:32 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


/home/mahajan.d/.conda/envs/ai-ehr/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


RuntimeError: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

In [22]:
model

NameError: name 'model' is not defined

In [13]:
SamplingParams?

Init signature:
SamplingParams(
    n: int = 1,
    best_of: Optional[int] = None,
    presence_penalty: float = 0.0,
    frequency_penalty: float = 0.0,
    repetition_penalty: float = 1.0,
    temperature: float = 1.0,
    top_p: float = 1.0,
    top_k: int = -1,
    min_p: float = 0.0,
    seed: Optional[int] = None,
    use_beam_search: bool = False,
    length_penalty: float = 1.0,
    early_stopping: Union[bool, str] = False,
    stop: Union[str, List[str], NoneType] = None,
    stop_token_ids: Optional[List[int]] = None,
    include_stop_str_in_output: bool = False,
    ignore_eos: bool = False,
    max_tokens: Optional[int] = 16,
    min_tokens: int = 0,
    logprobs: Optional[int] = None,
    prompt_logprobs: Optional[int] = None,
    detokenize: bool = True,
    skip_special_tokens: bool = True,
    spaces_between_special_tokens: bool = True,
    logits_processors: Optional[List[Callable[[List[int], torch.Tensor], torch.Tensor]]] = None,
    truncate_prompt_tokens: Optional[A

In [16]:
def generate_with_model(instruction, context_input, model):
    # Generate the prompt using the instruction and context
    prompt = generate_prompt(instruction, context_input)

    # Setup sampling parameters for generation
    # Here we only calculate length to control the output length dynamically
    context_length = len(tokenizer(prompt, return_tensors="pt").input_ids[0])
    print(context_length)
    stop_str = "--- end of response"
    max_input_length = min(2048, tokenizer.model_max_length - context_length)
    sampling_params = SamplingParams(temperature=0.01, max_tokens=max_input_length, stop=stop_str,)

    # Generate the response using the model
    outputs = model.generate(prompt, sampling_params)
    response_text = outputs[0].outputs[0].text  # Adjust based on actual output structure

    # outputs = llm.generate(final_prompt, sampling_params)
    # answer_texts = [output.outputs[0].text for output in outputs]
    
    # # Extract the "Response" part from the generated text
    # response = response_text.split("### Response:")[1].strip()
    # response = response_text.split("--- end of response")[1].strip() 
    
    # Print and return the response
    print(response_text)
    return response_text

In [17]:
# extract HPI summary USE THIS
def generate_bhc(hadm_id):
    # topic = '''
    # The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    # from a patient's hospital admission. 
    # You will be provided with input text and data from a specific patient's medical records and clinical notes
    # delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    # Answer clearly and concisely.
    # '''.replace('\n', '')
    topic = '''
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    from a patient's hospital admission. 
    You will be provided with input text and data from a specific patient's medical records and clinical notes 
    delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    '''.replace('\n', '').replace('    ', '')
    
    prompt = '''\n
    Write the text for the "brief hospital course" section of a discharge summary note 
    based on the input text using the following template:
    
    Brief Hospital Course
    The patient with a history of [briefly describe the relevent medical history or existing medical problems] 
    presents with [briefly summarize the patient's symptoms]. They were admitted to the [what unit, service or floor were they admitted to?]
    for [briefly summarize the reason for admission, initial diagnosis, and any major procedures or treatments].
    [Briefly summarize the discharge diagnosis at discharge]
    
    Problem-based hospital course---
    # [Medical condition 1] --- [briefly describe presentation and severity for this medical condition or symptom].
    [Summarize the related diagnostic work up and notable results]. 
    [Summarize the related medical or surgical treatment course].

    # [Medical condition 2] -- [briefly describe presentation and severity for this medical condition or symptom].
    [Summarize the diagnostic work up and notable results related to this medical condition]. 
    [Summarize the related medical or surgical treatment course]. 

    --- end of response

    Based on the input text, answer these questions about the patient's hospital admission:
    What are the patient's major medical conditions?
    Why was the patient admitted to the hospital?
    What symptoms did the patient present with? What was the severity and duration of the presenting symptoms?
    What treatments and procedures were administered and for what symptoms or medical conditions?
    What unit or floor were they admitted to?
    What was the diagnostic work up? What were the notable results?
    What medications were used to treat the patient? Were any medications new or discontinued?
    Are there changes to the existing medications?
    What was the patient diagnosed with at discharge?
    What was the patient initially diagnosed with?
    
    Fill in the blanks:
    Use the answers to the questions above to fill in the blanks.
    If you do not know the answer, fill in the blank with "___".
    Answer clearly and concisely. Do not include information about the review of systems.
    Stop generating when you reach "--- end of response"
    Do not repeat the instructions in your response or make comments after your response.
    '''.replace('    ', '')
    
    # Using the following input data as context, 
    # Summarize the patient's hospital course in the following way --
    # Start with brief description of the patient's major medical conditions and why they were admitted to the hospital.
    # Then, in a new section, outline the details of the treatment course and events of the hospital stay, 
    # focusing on the relevant diagnosis, medical conditions,
    # diagnostic work up, relevant results, procedures, medications that were used, and changes to baseline medication therapies. 
    # Organize the outline according to the major medical conditions and diagnosis.
    # Limit your response to 1024 words or less.
    # '''.replace('\n', '')
    # prompt = '''
    # Using the following patient specific data, 
    # briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.
    # '''.replace('\n', '')
    
    context_str =''
    df = discharge_sections_df
    cur_df = df[df['hadm_id'] == hadm_id]
    cols = ['CC']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = f"The chief complaint was --- {context_data} \n"
    cols = ['HPI']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The History of present illness --- \n{context_data} \n"
    # cols = ['Medication Lists']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The admission and discharge medication lists:\n{context_data} \n"
    # cols = ['Pertinent Results']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The Pertinent Results were:\n{context_data} \n"
    cols = ['Major Surgical Procedure']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The major procedures were --- \n{context_data} \n"
    cols = ['Discharge Diagnosis']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"At discharge, the patient was diagnosed with --- \n{context_data} \n"
    context_str = context_str.replace(":", '---').replace("==", '')
    
    df = diagnosis_df
    cols = ['icd_title']
    cur_df = df[df['hadm_id'] == hadm_id]
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
    context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles --- \n{context_data}\n "
    
    
    instruction = f"{topic} \n {prompt}"
    context_input = f"<text> {context_str}</text> "
    
    response = generate_with_model(instruction, context_input, model)
    return response
    
# # extract HPI summary USE THIS
# def generate_bhc(hadm_id):
#     topic = '''
#     The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
#     from a patient's hospital admission. 
#     You will be provided with text and data from a specific patient's medical records and clinical notes
#     delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
#     Answer clearly and concisely.
#     '''.replace('\n', '')
    
#     # prompt = '''
#     # Using the following patient specific data, 
#     # briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.
#     # Then, create a list of the patient's acute and chronic medical conditions and the associated treatments course.
#     # '''.replace('\n', '')
#     prompt = '''
#     Using the following patient specific data, 
#     briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.
#     '''.replace('\n', '')
    
#     context_str =''
#     df = discharge_sections_df
#     cur_df = df[df['hadm_id'] == hadm_id]
#     cols = ['CC']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = f"The chief complaint was - {context_data} \n"
#     cols = ['HPI']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = context_str + f"The History of present illness: \n{context_data} \n"
#     # cols = ['Medication Lists']
#     # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     # context_str = context_str + f"The admission and discharge medication lists:\n{context_data} \n"
#     # cols = ['Pertinent Results']
#     # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     # context_str = context_str + f"The Pertinent Results were:\n{context_data} \n"
#     cols = ['Major Surgical Procedure']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = context_str + f"The major procedures were:\n{context_data} \n"
#     cols = ['Discharge Diagnosis']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = context_str + f"The patient was diagnosed with:\n{context_data} \n"
    
#     df = diagnosis_df
#     cols = ['icd_title']
#     cur_df = df[df['hadm_id'] == hadm_id]
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
#     context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "
    
    
#     instruction = f"{topic} \n Instruction: {prompt}"
#     context_input = f"<text> Text:{context_str}</text> "
    
#     response = generate_with_model(instruction, context_input, model)
#     return response

In [18]:
def generate_discharge_instructions(hadm_id):
    # topic = '''
    # The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    # from a patient's hospital admission. 
    # You will be provided with input text and data from a specific patient's medical records and clinical notes
    # delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    # Answer clearly and concisely.
    # '''.replace('\n', '')
    
    # prompt = '''
    # Given the following input data from a patient's hospital stay,
    # Compose a letter to the patient that includes answers to the following questions.
    # What was the patient admitted for? What were the major treatments and exam findings?
    # What is the recommended follow-up plan or instructions?
    # Are there any changes to the patient's existing medication, 
    # and if so, what are the new medications, dose changes, or medications that should be stopped?
    # The letter should be courteous, easy to understand, and it should be written in layman's language. 
    # Limit the use of medical jargon and acronyms.
    # Limit your response to 500 words or less.
    # '''.replace('\n', '')

    topic = '''
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    from a patient's hospital admission. 
    You will be provided with input text and data from a specific patient's medical records and clinical notes 
    delimited by xml tags <text> </text>.  
    '''.replace('\n', '').replace('    ', '')
    
    prompt = '''\n
    Write a brief and curteous letter to the patient that summarizes their hospital stay and communicates 
    follow-up instructions as well as important changes to their medications. Use the 
    following template:
    
    Discharge Instructions ---
    Dear ____,

    It was a pleasure to take care of you during your recent hospital admission. 
    You were admitted to the hospital because [explain the reason for admission]. 
    [Briefly explain the diagnostic work up and explain the results]
    During your hospital stay your treatments included [briefly explain the major treatments and procedures].

    [Briefly explain ongoing issues and follow-up recommendations].

    [Briefly explain any new medications and changes to medications].

    If your symptoms return, please call our office (555-555-5555) or 911 for immedicate assistance

    Sincerely,
    Your Team___
    
    --- end of response

    Questions to answer based on the input text:
    Why was the patient admitted to the hospital?
    What treatments and procedures were administered and for what symptoms or medical conditions?
    What was the diagnostic work up related to the chief complaint? What were the notable results?
    What medications were used to treat the patient? Were any medications new or discontinued?
    Are there changes to the existing medications?
    What are the ongoing issues and follow-up recommendations?
    
    Fill in the blanks:
    Use the input data and answers to the questions above to fill in the blanks.
    If you do not know the answer, fill in the blank with "___".
    Answer clearly and concisely. Use easy to understand language or layman's terms.
    Address the patient as "you". Do not refer to the patient in the third person.
    Minimize the use of medical jargon and acronyms.
    Stop generating when you reach "--- end of response".
    Do not repeat the instructions in your response.
    '''.replace('    ', '')
    
    context_str = ''
    df = discharge_sections_df
    cur_df = df[df['hadm_id'] == hadm_id]
    cols = ['CC']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = f"The chief complaint was --- {context_data} \n"
    cols = ['HPI']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = f"The history of present illness was --- {context_data} \n"
    # cols = ['Medication Lists']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The admission and discharge medication list --- \n{context_data} \n"
    cols = ['Transitional Issues']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The ongoing issues and followup instructions include ---  \n{context_data} \n"
    # cols = ['BHC']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The Brief Hospital course was:\n{context_data} \n"
    cols = ['Discharge Diagnosis']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"At discharge, the patient was diagnosed with --- \n{context_data} \n"
    context_str = context_str.replace(":", '---').replace("==", '')
    
    # df = diagnosis_df
    # cols = ['icd_title']
    # cur_df = df[df['hadm_id'] == hadm_id]
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
    # context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles --- \n{context_data}\n "
    
    instruction = f"{topic} \n {prompt}"
    context_input = f"<text> {context_str}</text> "
    
    response = generate_with_model(instruction, context_input, model)
    return response
    
# def generate_discharge_instructions(hadm_id):
#     topic = '''
#     The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
#     from a patient's hospital admission. 
#     You will be provided with text and data from a specific patient's medical records and clinical notes
#     delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
#     Answer clearly and concisely.
#     '''.replace('\n', '')
    
#     prompt = '''
#     Given the following information from a patient's hospital stay,
#     Compose a letter to the patient that is courteous and easy to understand. 
#     There should be limited medical jargon and it should be written in layman's language.  
#     The letter will briefly summarize the reason for admission, the treatment course, relevant tests and results.  
#     It will also summarize any changes to the patient's current medical management including medication changes. 
#     Is there any scheduled or recommended follow up?
#     If there was a surgery, what are the post-operative or wound care instructions?
#     Limit the letter to 3 paragraphs.
#     '''.replace('\n', '')
    
#     context_str = ''
#     df = discharge_sections_df
#     cur_df = df[df['hadm_id'] == hadm_id]
#     cols = ['HPI']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = f"The history of present illness was: {context_data} \n"
#     # cols = ['Medication Lists']
#     # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     # context_str = context_str + f"The admission medication list: \n{context_data} \n"
#     cols = ['Transitional Issues']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = context_str + f"The ongoing issues to be addressed after discharge: \n{context_data} \n"
#     # cols = ['BHC']
#     # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     # context_str = context_str + f"The Brief Hospital course was:\n{context_data} \n"
#     df = diagnosis_df
#     cols = ['icd_title']
#     cur_df = df[df['hadm_id'] == hadm_id]
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
#     context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "
    
    
#     instruction = f"{topic} \n Instruction: {prompt}"
#     context_input = f"<text> Text:{context_str}</text> "
    
#     response = generate_with_model(instruction, context_input, model)
#     discharge_instructions = response
#     return discharge_instructions


In [19]:
import pandas as pd
import time
# Create an empty results table
results = pd.DataFrame(columns=['hadm_id', 'discharge_instructions'])

# results path:
results_path = '../discharge_me/data/scoring/submission_01/'
file_name = 'results_discharge_instructions.csv'
results_file_path = os.path.join(results_path, file_name)

count = 0
total_records = len(target_df.hadm_id)
for hadm_id in target_df['hadm_id'][:]:  # Replace with your actual hadm_id values
    count += 1
    percent_complete = count*100/total_records
    print(f"{count} of {total_records}, {percent_complete}% hadm_id: {hadm_id} **********\n\n")
    # brief_hospital_course = generate_bhc(hadm_id)
    start_time_di = time.time()
    discharge_instructions = generate_discharge_instructions(hadm_id)
    new_row = pd.DataFrame({'hadm_id': [hadm_id], 'discharge_instructions': [discharge_instructions]})
    results = pd.concat([results, new_row], ignore_index=True)
    end_time_di = time.time()
    duration_di = end_time_di - start_time_di
    print(f"Time taken for generate_discharge_instructions: {duration_di} seconds.")
    # results = results.append({'hadm_id': hadm_id, 'brief_hospital_course': brief_hospital_course, 'discharge_instructions': discharge_instructions}, ignore_index=True)
    results.to_csv(results_file_path, index=False)

1 of 10962, 0.009122422915526365% hadm_id: 24962904 **********




NameError: name 'model' is not defined

In [20]:
import time

# Patient ID to be processed
patient_id = 24962904

# Timing generate_bhc function
start_time_bhc = time.time()
brief_hospital_course = generate_bhc(patient_id)
end_time_bhc = time.time()
duration_bhc = end_time_bhc - start_time_bhc
print(f"Time taken for generate_bhc: {duration_bhc} seconds.")

# Timing generate_discharge_instructions function
start_time_di = time.time()
discharge_instructions = generate_discharge_instructions(patient_id)
end_time_di = time.time()
duration_di = end_time_di - start_time_di
print(f"Time taken for generate_discharge_instructions: {duration_di} seconds.")


NameError: name 'model' is not defined